In [1]:
import ipywidgets as widgets
from ipywidgets import interact
import matplotlib.pyplot as plt
import json
import pprint
import glob
from tabulate import tabulate

In [8]:
def plotting_coordinates(info, nb_events, team_away, team_home):
    @interact
    def interact_event(eventIdx=(0, nb_events-1, 1)):
        # Retrieve event details
        event = info['plays'][eventIdx]
        coord = {'x' : event['details']['xCoord'], 'y' : event['details']['yCoord']}
        period = event['periodDescriptor']['number']
        print(f'{eventIdx} de {event}')
        # Determine rink side for the event
        defending_side = event.get('homeTeamDefendingSide', 'unknown')
        if defending_side == 'right':
            teams_side = f"{team_away}{' ' * 80}{team_home}"
        elif defending_side == 'left':
            teams_side = f"{team_home}{' ' * 80}{team_away}"
        else:
            teams_side = "Unknown sides"

        # Plot the coordinates on the rink
        im = plt.imread('/mnt/c/Users/mouaa/Desktop/github/NHL/figures/nhl_rink.png')
        fig, ax = plt.subplots(figsize=(20, 15))
        height_rink, width_rink = 85, 200  # NHL rink dimensions
        ax.imshow(im, extent=[-width_rink/2, width_rink/2, -height_rink/2, height_rink/2])
        if coord:
            ax.scatter(coord['x'], coord['y'], s=300, c='blue')
        plt.xlabel('feet')
        plt.ylabel('feet')
        plt.title(
            f"{event['typeDescKey']}\n"
            f"{event['timeInPeriod']} P-{period}\n"
            f"{teams_side}",
            fontsize=15
        )

        plt.show()
        pprint.pprint(event)


In [3]:
def get_match_info(info, gameId=None, round=None, rencontre=None, match=None):
    # Calculate SoG (Shots on Goal)
    SoG_home, SoG_away = 0, 0
    for play in info['plays']:
        if play['typeDescKey'] == 'shot-on-goal':
            #if play['team']['id'] == info['gameData']['teams']['home']['id']:
            #    SoG_home += 1
            #elif play['team']['id'] == info['gameData']['teams']['away']['id']:
            #    SoG_away += 1
            SoG_home = play['details']['homeSOG']
            SoG_away = play['details']['awaySOG']
    # Display game info
    print(info['gameDate'])
    team_away = info['awayTeam']['commonName']['default']
    team_home = info['homeTeam']['commonName']['default']

    if gameId is not None:
        print(f"Game ID: {gameId}; {team_away} (away) vs {team_home} (home) \n")
    else:
        print(f"Playoffs rounds: {round} | rencontre: {rencontre} | match: {match}; {team_away} (away) vs {team_home} (home) \n")

    if len(info['plays']) == 0:
        print(f'The playoff round opposing {team_home} and {team_away} was finished in less than {match}')
        played = False
    else:
        # Handle missing shootoutInfo
        #shootout_info = info['liveData']['linescore'].get('shootoutInfo', {})
        #shootout_home = shootout_info.get('home', {'scores': 0, 'attempts': 0})
        #shootout_away = shootout_info.get('away', {'scores': 0, 'attempts': 0})

        table = [
            ['', 'Home', 'Away'],
            ['Teams: ', team_home, team_away],
            ['Goals: ', info['homeTeam']['score'], info['awayTeam']['score']],
            ['SoG: ', SoG_home, SoG_away],
            ['SO Goals: ', None, None],
            ['SO Attemps: ', None, None]
        ]
        print(tabulate(table, headers='firstrow', tablefmt='fancy_grid'))
        played = True

    return team_away, team_home, played


In [6]:
@interact
def interactive_debugger(season=['2016_2017', '2017_2018', '2018_2019', '2019_2020', '2020_2021'], season_type=['regular', 'playoff']):
    """
    Interactive debugger for visualizing NHL games.
    """
    year = season.split("_")[0]

    # Regular season
    if season_type == 'regular':
        season_code = '02'
        nb_games = len(glob.glob(f'/mnt/c/Users/mouaa/Desktop/github/NHL/data/raw/{season}/{year+season_code}*'))

        # Widget with Game ID
        @interact
        def interact_game(gameId=(0, nb_games-1, 1)):
            game_number = str(int(gameId) + 1).zfill(4)
            json_path = f'/mnt/c/Users/mouaa/Desktop/github/NHL/data/raw/{season}/{year + season_code + game_number}.json'
            
            try:
                with open(json_path, 'r') as file:
                    info = json.load(file)

                # Validate the presence of 'allPlays'
                if not info['plays']:
                    print(f"No events found for game {game_number}.")
                    return
                
                nb_events = len(info['plays'])
                # Print match stats
                team_away, team_home, _ = get_match_info(info, gameId)
                # Call plotting function
                plotting_coordinates(info, nb_events, team_away, team_home)
            except FileNotFoundError:
                print(f"No file found for game: {game_number}")
            except KeyError as e:
                print(f"Missing key in JSON: {e}")
            except Exception as e:
                print(f"Error processing game {game_number}: {e}")

    # Playoffs
    else:
        @interact
        def interact_game(round=(1, 4, 1)):
            @interact
            def interact_meeting(rencontre=(1, int(0.5**round*16), 1), match=(1, 7, 1)):
                json_path = f'/mnt/c/Users/mouaa/Desktop/github/NHL/data/raw/{season}/{year + "030" + str(round) + str(rencontre) + str(match)}.json'
                
                try:
                    with open(json_path, 'r') as file:
                        info = json.load(file)

                    # Validate the presence of 'allPlays'
                    if not info['plays']:
                        print(f"No events found for round {round}, rencontre {rencontre}, match {match}.")
                        return

                    nb_events = len(info['plays'])
                    # Print match stats
                    team_away, team_home, played = get_match_info(info, round=round, rencontre=rencontre, match=match)
                    # Call plotting function
                    if played:
                        plotting_coordinates(info, nb_events, team_away, team_home)
                except FileNotFoundError:
                    print(f"No file found for round: {round}; rencontre: {rencontre}; match: {match}")
                except KeyError as e:
                    print(f"Missing key in JSON: {e}")
                except Exception as e:
                    print(f"Error processing round {round}, rencontre {rencontre}, match {match}: {e}")


interactive(children=(Dropdown(description='season', options=('2016_2017', '2017_2018', '2018_2019', '2019_202…